In [1]:
import pandas as pd
from folium import IFrame
import folium
import base64

Reading the txt file coitaining the INTERMAGNET observatories informations.

In [2]:
df_RMS = pd.read_csv('imos_rms.txt',
                     sep = ' ',
                     index_col = [0]
                    )

A quick look at the dataframe, it's import to verify if the structure is correct.

In [11]:
df_RMS

,X_RMS,Y_RMS,Z_RMS,Latitude,Longitude
ABG,5.382,6.568,5.360,18.638,72.872
API,5.702,6.206,6.418,-13.815,-171.781
ABK,11.237,2.585,5.988,68.358,18.823
ASC,3.924,5.849,3.682,-7.950,-14.380
BDV,2.287,1.391,2.624,49.080,14.020
...,...,...,...,...,...
WNG,1.815,2.449,1.628,53.725,9.053
YAK,6.071,4.522,6.144,61.960,129.660
DMC,14.200,7.726,20.567,-75.250,124.167
LZH,4.995,5.414,3.534,36.087,103.845


As the map is part of my Master's project. I'm plotting only the observatories with good RMSe values (this is part of other analysis :) ). So, let's filter the dataframe, removing RMSe values greater than 15.

In [3]:
df_RMS = df_RMS.drop(df_RMS.loc[df_RMS['X_RMS'] > 15].index)
df_RMS = df_RMS.drop(df_RMS.loc[df_RMS['Y_RMS'] > 15].index)
df_RMS = df_RMS.drop(df_RMS.loc[df_RMS['Z_RMS'] > 15].index)

Note that now We only have 93 rows in the dataframe.

In [28]:
df_RMS

,X_RMS,Y_RMS,Z_RMS,Latitude,Longitude
ABG,5.382,6.568,5.360,18.638,72.872
API,5.702,6.206,6.418,-13.815,-171.781
ABK,11.237,2.585,5.988,68.358,18.823
ASC,3.924,5.849,3.682,-7.950,-14.380
BDV,2.287,1.391,2.624,49.080,14.020
...,...,...,...,...,...
VSS,3.372,2.765,1.965,-22.400,-43.650
WNG,1.815,2.449,1.628,53.725,9.053
YAK,6.071,4.522,6.144,61.960,129.660
LZH,4.995,5.414,3.534,36.087,103.845


In [6]:
map = folium.Map(location=[0, 0], zoom_start=3, tiles = 'cartodbpositron', title = 'INTERMAGNET observatories SV')


fgmm = folium.FeatureGroup(name = 'Filtered Monthly means')
for Latitude, Longitude, Imos in zip(df_RMS['Latitude'],df_RMS['Longitude'], df_RMS.index):
    station = Imos
    tooltip = str(Imos)
    icon = folium.Icon(color="black", icon="ok")
    encoded = base64.b64encode(open(f'RMS_figures/{station}_RMS_monthly_mean_MAP.jpeg', 'rb').read()).decode()
    html = '<img src="data:image/png;base64,{}">'.format
    resolution, width, height = 65, 8, 6
    iframe = IFrame(html(encoded), width=(width*resolution)+20, height=(height*resolution)+20)
    popup = folium.Popup(iframe, max_width=2650)
    #m = folium.Map(location=[lat, lon], zoom_start=5)
    #lon, lat = Longitude, Latitude
    fgmm.add_child(folium.CircleMarker(location=[Latitude, Longitude],
                                       tooltip= tooltip,
                                       popup=popup,
                                       icon = icon,
                                       radius = 7,
                                       color = 'red',
                                       fill = True
                                      )
                  )


fgsv = folium.FeatureGroup(name = 'Filtered Secular Variation')
for Latitude, Longitude, Imos in zip(df_RMS['Latitude'],df_RMS['Longitude'], df_RMS.index):
    station = Imos
    tooltip = str(Imos)
    icon = folium.Icon(color="black", icon="ok")
    encoded = base64.b64encode(open(f'RMS_figures/{station}_RMS_MAP.jpeg', 'rb').read()).decode()
    html = '<img src="data:image/png;base64,{}">'.format
    resolution, width, height = 65, 8, 6
    iframe = IFrame(html(encoded), width=(width*resolution)+20, height=(height*resolution)+20)
    popup = folium.Popup(iframe, max_width=2650)
    #m = folium.Map(location=[lat, lon], zoom_start=5)
    #lon, lat = Longitude, Latitude
    fgsv.add_child(folium.CircleMarker(location=[Latitude, Longitude],
                                       tooltip= tooltip,
                                       popup=popup,
                                       icon = icon,
                                       radius = 7,
                                       fill = True
                                      )
                  )
    

   
fgsvn = folium.FeatureGroup(name = 'Not filtered Secular Variation')
for Latitude, Longitude, Imos in zip(df_RMS['Latitude'],df_RMS['Longitude'], df_RMS.index):
    station = Imos
    tooltip = str(Imos)
    icon = folium.Icon(color="black", icon = 'ok')
    encoded = base64.b64encode(open(f'RMS_figures/{station}_RMS_no_correction_MAP.jpeg', 'rb').read()).decode()
    html = '<img src="data:image/png;base64,{}">'.format
    resolution, width, height = 65, 8, 6
    iframe = IFrame(html(encoded), width=(width*resolution)+20, height=(height*resolution)+20)
    popup = folium.Popup(iframe, max_width=2650)
    fgsvn.add_child(folium.CircleMarker(location=[Latitude,
                                                  Longitude
                                                  ],
                                        tooltip= tooltip,
                                        popup=popup,
                                        icon = icon,
                                        radius = 7,
                                        color = 'black',
                                        fill = True
                                       )
                   )
    
map.add_child(fgmm)
map.add_child(fgsv)
map.add_child(fgsvn)
map.add_child(folium.LayerControl())
map
#map.save('Interactive_map_layer.html')